In [1]:
from coffea.util import load
import numpy as np
import uproot3
from tqdm import tqdm
from pprint import pprint
import gc

#path = '/eos/user/k/kmotaama/MuOniaRun17E/'
#file = path + 'MuOniaRun17E.coffea'
    
#acc = load(file)
#acc

In [14]:
import os

ERAS = ['B', 'C', 'D', 'E', 'F']
ERAS = ['F']

path = '/afs/cern.ch/work/m/mabarros/public/CMSSW_10_6_12/src/OniaOpenCharmRun2ULAna/output/Charmonium_2017_Nano_Coffea/'
save_path = "/afs/cern.ch/work/m/mabarros/public/CMSSW_10_6_12/src/OniaOpenCharmRun2ULAna/fit/Data_to_fit/"

files = {}

for era in ERAS:
    with os.scandir(path + f'Run{era}') as it:
        file_array = []
        for file in it:
            if file.name.endswith('.coffea') and file.name.find('hists') == -1:
                file_array.append(file.path)
        files[era] = file_array

In [6]:
def load_acc(file_list):
    acc = load(file_list[0])
    for i in tqdm(file_list[1:], desc="Loading", unit=" files", total=len(file_list)-1):
        acc += load(i)
    return acc

In [15]:
def create_ttree(acc, path, name):
 
    ## Jpsi mass and decay length

    # Mass
    jpsi_mass = acc['Dimu']['mass'].value[acc['Dimu']['is_jpsi'].value]
    # Decay length
    jpsi_dl = acc['Dimu']['dl'].value[acc['Dimu']['is_jpsi'].value]
    
    ## Dstar    
    all_data_dstar = acc['Dstar']['deltamr'].value[~acc['Dstar']['wrg_chg'].value]
    cut = all_data_dstar[(all_data_dstar >= 0.14) & (all_data_dstar <= 0.16)]

    dstar_mass = cut[:]

    ## Dzero    
    all_data_dzero_12 = acc['D0']['mass12'].value
    all_data_dzero_21 = acc['D0']['mass21'].value
    cut_12 = all_data_dzero_12[(all_data_dzero_12 > 1.75) & (all_data_dzero_12 < 1.96)]
    cut_21 = all_data_dzero_21[(all_data_dzero_21 > 1.75) & (all_data_dzero_21 < 1.96)]

    dzero_mass_12 = cut_12[:]
    dzero_mass_21 = cut_21[:]
    
    ## TTree creation
    
    # Jpsi: mass and decay length
    #with uproot3.recreate(path + name + "jpsi.root") as f:
        #f['Jpsi'] = uproot3.newtree({"mass": "float64", "jpsi_dl" : "float32"})
        #f['Jpsi'].extend({"mass": jpsi_mass, "jpsi_dl": jpsi_dl})
        
   # Dzero - mass12
    #with uproot3.recreate(path + name + "_D0_mass12.root") as f:
        #f['D012'] = uproot3.newtree({"mass": "float64"})
        #f['D012'].extend({"mass": dzero_mass_12})
    # Dzero - mass 21
    #with uproot3.recreate(path + name + "_D0_mass21.root") as f:
        #f['D021'] = uproot3.newtree({"mass": "float64"})
        #f['D021'].extend({"mass": dzero_mass_21})
    # Dstar - deltamr
    #with uproot3.recreate(save_path + name + "_Dstar_deltamr.root") as f:
        #f['Dstar'] = uproot3.newtree({"deltam": "float64"})
        #f['Dstar'].extend({"deltam": dstar_mass})
    
    ## Association between Jpsi and Dstar
            
    # Dstar
    wrg_chg = acc['DimuDstar']['Dstar']['wrg_chg'].value
    Dstar = acc['DimuDstar']['Dstar']['deltamr'].value[~wrg_chg]

    # Jpsi
    Jpsi = acc['DimuDstar']['Dimu']['mass'].value[~wrg_chg]

    with uproot3.recreate(path + name + "_JpsiDstar.root") as ds:
        ds['JpsiDstar'] = uproot3.newtree({"dstar_mass": "float32", "jpsi_mass": "float32"})
        ds['JpsiDstar'].extend({"dstar_mass": Dstar, "jpsi_mass": Jpsi})

In [16]:
for era in files:
    print(f"Loading era {era}:")
    acc = load_acc(files[era])
    print("Creating TTrees at: "+ save_path)
    name = "Charmonium2017" + era
    create_ttree(acc, path, name)
    del acc
    gc.collect()


Loading era F:


Loading: 100%|███████████████| 90/90 [01:08<00:00,  1.32 files/s]


Creating TTrees at: /afs/cern.ch/work/m/mabarros/public/CMSSW_10_6_12/src/OniaOpenCharmRun2ULAna/fit/Data_to_fit/


In [14]:
############## BACKUP #################

JpsiDstar = acc['DimuDstar']
    # Jpsi mass
    Jpsi = JpsiDstar['Dimu']['mass'].value[JpsiDstar['Dimu']['is_jpsi'].value & ~JpsiDstar['Dstar']['wrg_chg'].value]
    # Dstar - deltamr
    Dstar = acc['DimuDstar']['Dstar']['deltamr'].value
    # 
    with uproot3.recreate(path + name + "_JpsiDstar.root") as f:
        f['JpsiDstar'] = uproot3.newtree({"jpsi_mass": "float64", "dstar_deltmass": "float64"})
        f['JpsiDstar'].extend({"jpsi_mass": Jpsi, "dstar_deltmass": Dstar})

IndentationError: unexpected indent (2408270560.py, line 5)